In [1]:
import ciftify
from ciftify.utils import run
import os
import datetime
import logging
import glob

In [2]:
#working_dir = '/home/edickie/Documents/ciftify_tests/'
working_dir = '/scratch/edickie/ciftify_intergration_tests/'
src_data_dir= os.path.join(working_dir,'src_data')
#work_from = '/home/edickie/Documents/ciftify_tests/run_2017-08-26/'
work_from = None

if work_from:
    new_outputs = work_from
else:
    new_outputs= os.path.join(working_dir,'run_{}'.format(datetime.date.today()))
DEBUG = True
DRYRUN = True

## getting the data

freesurfer_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_freesurfer_sub50004-50008.zip'
func_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_sub50004-50008.zip'

#subids = ['sub-50005','sub-50007']
subids = ['sub-50005','sub-50006']

logger = logging.getLogger('ciftify')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)    

fs_subjects_dir = os.path.join(src_data_dir, 'ds000030_R1.0.4',
                               'derivatives','freesurfer')
hcp_data_dir = os.path.join(new_outputs, 'hcp')

if not os.path.exists(hcp_data_dir):
    run(['mkdir','-p',hcp_data_dir])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


In [3]:
def download_file(web_address, local_filename):
    '''download file if it does not exist'''
    if not os.path.isfile(local_filename):
        run(['wget', web_address, '-O', local_filename])
    if not os.path.getsize(local_filename) > 0:
        os.remove(local_filename)

In [4]:
def get_recon_all_outputs(hcp_data_dir, subid):
    recon_all_out = folder_contents_list(os.path.join(hcp_data_dir, subid))
    recon_all_out = [x.replace(subid, 'subid') for x in recon_all_out]
    return(recon_all_out)

In [5]:
if not os.path.exists(src_data_dir):
    run(['mkdir','-p',src_data_dir])

for subid in subids:
    sub_fs_path = os.path.join('ds000030_R1.0.4','derivatives','freesurfer', subid)
if not os.path.exists(os.path.join(src_data_dir, sub_fs_path)):
    if not os.path.exists(os.path.join(src_data_dir, 
                                       os.path.basename(freesurfer_webtgz))):        
        run(['wget', '-P', src_data_dir, freesurfer_webtgz])

    run(['unzip', 
         os.path.join(src_data_dir, 
                      os.path.basename(freesurfer_webtgz)), 
         os.path.join(sub_fs_path,'*'),
         '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

In [6]:
def folder_contents_list(path):
    '''returns a list of folder contents'''
    folder_contents = glob.glob(os.path.join(path, '**'), recursive = True)
    folder_contents = [x.replace('{}/'.format(path),'') for x in folder_contents ]
    folder_contents = folder_contents[1:] ## the first element is the path name
    return(folder_contents)

In [7]:
run(['ciftify_recon_all', '--resample-LowRestoNative', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[0]])

Running: ciftify_recon_all --resample-LowRestoNative --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50005


0

In [8]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[1]])

Running: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50006


0

In [9]:
for subid in subids:
    run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_recon_all snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp sub-50005
Running: cifti_vis_recon_all snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp sub-50006
Running: cifti_vis_recon_all index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


0

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
#         '--fs-subjects-dir', fs_subjects_dir, 
#         subid])

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
# run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

# Getting ABIDE and running recon-all

In [10]:
abide_amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1'

subid = 'NYU_0050954'
abide_freesurfer = os.path.join(src_data_dir, 'abide', 'freesurfer')

fs_subdir = os.path.join(abide_freesurfer, subid)
if not os.path.exists(fs_subdir):    
    run(['mkdir', '-p', fs_subdir])

for subdir in ['mri', 'surf','label','scripts']:
    localdir = os.path.join(fs_subdir, subdir)
    if not os.path.exists(localdir):
        run(['mkdir', '-p', localdir])
    
for filename in ['T1.mgz', 'aparc+aseg.mgz', 'aparc.a2009s+aseg.mgz', 'wmparc.mgz', 'brain.finalsurfs.mgz']:
    download_file(os.path.join(abide_amazon_addy, subid, 'mri', filename.replace('+','%20')),
                               os.path.join(fs_subdir, 'mri', filename))

for surface in ['pial', 'white', 'sphere.reg', 'sphere', 'curv', 'sulc', 'thickness']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'surf', '{}.{}'.format(hemi,surface)),
                      os.path.join(fs_subdir, 'surf', '{}.{}'.format(hemi,surface)))

for labelname in ['aparc', 'aparc.a2009s', 'BA']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'label', '{}.{}.annot'.format(hemi,labelname)),
                      os.path.join(fs_subdir, 'label', '{}.{}.annot'.format(hemi,labelname))) 

for script in ['recon-all.done', 'build-stamp.txt']:
    download_file(os.path.join(abide_amazon_addy, subid, 'scripts', script), 
                  os.path.join(fs_subdir, 'scripts', script))

In [11]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
        '--fs-subjects-dir', abide_freesurfer, 
        'NYU_0050954'])

Running: ciftify_recon_all --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --fs-subjects-dir /scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer NYU_0050954
/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc.a2009s+aseg.mgz not found
/scratch/edickie/ciftify_intergration_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz not found



0

In [12]:

run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, 'NYU_0050954'])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_recon_all snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp NYU_0050954
Running: cifti_vis_recon_all index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


0

## Download ABIDE PCP data for ciftify_vol_result tests

In [ ]:


def download_vmhc(subid):
    amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/vmhc/{}_vmhc.nii.gz'.format(subid)
    sub_vmhc = os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subid))
    download_file(amazon_addy, sub_vmhc)        

src_vmhc = os.path.join(src_data_dir, 'abide', 'vmhc')    
if not os.path.exists(src_vmhc): 
    run(['mkdir', src_vmhc])

subjects=['NYU_0050954','NYU_0050955']
for subid in subjects:
    download_vmhc(subid)

### This is where I need to add the cifti_vis tests.. (with standard)

# Running ciftify_subject_fmri 

In [13]:
subids = ['sub-50005','sub-50006']
subid=subids[0]
native_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_preproc.nii.gz'.format(subid))
mat_file = os.path.join(src_data_dir, 'ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_T1_to_EPI.mat'.format(subid))

with open(mat_file, "w") as text_file:
    text_file.write('''1.03  -0.015  0.0025  -15.0
0.014  1.01  -0.005  -11.9
-0.007 0.01  0.99  2
0  0  0  1
''')

t1_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
if not os.path.exists(native_func):
    run(['flirt', '-in', t1_func, '-ref', t1_func, 
         '-out', native_func, '-init', mat_file, '-applyxfm'])

run(['ciftify_subject_fmri', '--SmoothingFWHM', '12', 
    '--hcp-data-dir', hcp_data_dir, 
    native_func, subid, 'rest_test1'])

Running: ciftify_subject_fmri --SmoothingFWHM 12 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test1


0

In [14]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--SmoothingFWHM', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test1', subid])

Running: cifti_vis_fmri snaps --SmoothingFWHM 12 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp rest_test1 sub-50005


0

In [15]:
subid=subids[0]
native_func_mean = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_mean.nii.gz'.format(subid))

run(['fslmaths', native_func, '-Tmean', native_func_mean])

run(['ciftify_subject_fmri', '--hcp-data-dir', hcp_data_dir, 
    '--FLIRT-template', native_func_mean, native_func, subid, 'rest_test2'])


Running: fslmaths /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz -Tmean /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_mean.nii.gz
Running: ciftify_subject_fmri --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --FLIRT-template /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_mean.nii.gz /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test2


0

In [16]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--smooth-conn', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test2', subid])

Running: cifti_vis_fmri snaps --smooth-conn 12 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp rest_test2 sub-50005


0

In [17]:
subid = subids[1]
run(['ciftify_subject_fmri', '--SmoothingFWHM', '8', '--FLIRT-dof', '30',
    '--hcp-data-dir', hcp_data_dir, 
     '--OutputSurfDiagnostics',
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid)),
    subid, 'rest_test_dof30'])

Running: ciftify_subject_fmri --SmoothingFWHM 8 --FLIRT-dof 30 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --OutputSurfDiagnostics /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50006/func/sub-50006_task-rest_bold_space-T1w_preproc.nii.gz sub-50006 rest_test_dof30


0

In [18]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_test_dof30', subid])

Running: cifti_vis_fmri snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp rest_test_dof30 sub-50006


0

In [19]:
subid = subids[1]
run(['ciftify_subject_fmri', '--already-in-MNI', '--SmoothingFWHM', '8', 
    '--hcp-data-dir', hcp_data_dir, 
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid)),
    subid, 'rest_bad_transform'])

Running: ciftify_subject_fmri --already-in-MNI --SmoothingFWHM 8 --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50006/func/sub-50006_task-rest_bold_space-T1w_preproc.nii.gz sub-50006 rest_bad_transform


0

In [20]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_bad_transform', subid])

Running: cifti_vis_fmri snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp rest_bad_transform sub-50006


0

In [21]:
run(['cifti_vis_fmri', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_fmri index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


0

In [22]:
from glob import glob
dtseries_files = glob(os.path.join(hcp_data_dir,'*',
                                       'MNINonLinear', 'Results',
                                       '*', '*Atlas_s*.dtseries.nii'))

for dtseries in dtseries_files:
    run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, dtseries, 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries)))))])
run(['cifti_vis_RSN', 'index', 
     '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test2/rest_test2_Atlas_s0.dtseries.nii sub-50005
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii sub-50005
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii sub-50005
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/Results/rest_bad_

0

# Running PINT and related functions

In [23]:
subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--pcorr', 
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])


Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50005
Running: ciftify_PINT_vertices --pcorr /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /projects/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50005/sub-50005


0

In [24]:
subid = subids[0]
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii sub-50005 /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50005/sub-50005_summary.csv
/projects/edickie/code/miniconda3/envs/ciftify_prereqs/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '



0

In [33]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006
Running: ciftify_PINT_vertices /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/fsaverage_LR32k/sub-50006.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/fsaverage_LR32k/sub-50006.R.midthickness.32k_fs_LR.surf.gii /projects/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006


0

In [34]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50006 /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006_summary.csv


0

In [27]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--outputall',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, '{}_all'.format(subid))])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006
Running: ciftify_PINT_vertices --outputall /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/fsaverage_LR32k/sub-50006.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/fsaverage_LR32k/sub-50006.R.midthickness.32k_fs_LR.surf.gii /projects/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006_all


0

In [28]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_all_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50006/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50006 /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006_all_summary.csv


0

In [35]:
run(['cifti_vis_PINT', 'index', '--hcp-data-dir',  hcp_data_dir])

Running: cifti_vis_PINT index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


0

In [36]:
summary_files = glob(os.path.join(new_outputs, 'PINT', '*','*_summary.csv'))
concat_cmd = ['ciftify_postPINT1_concat.py',  
           os.path.join(new_outputs, 'PINT', 'concatenated.csv')]
concat_cmd.extend(summary_files)
run(concat_cmd)

Running: ciftify_postPINT1_concat.py /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/concatenated.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50005/sub-50005_summary.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006_summary.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/sub-50006/sub-50006_all_summary.csv


0

In [37]:
run(['ciftify_postPINT2_sub2sub.py', os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances.csv')])

Running: ciftify_postPINT2_sub2sub.py /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/concatenated.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/ivertex_distances.csv


0

In [38]:
run(['ciftify_postPINT2_sub2sub.py', '--roiidx', '14',
     os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances_roiidx14.csv')])

Running: ciftify_postPINT2_sub2sub.py --roiidx 14 /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/concatenated.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/PINT/ivertex_distances_roiidx14.csv


0

# Running ciftify_surface_rois

In [39]:
vertices1_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices1_csv, "w") as text_file:
    text_file.write('''hemi,vertex
L,11801
L,26245
L,26235
L,26257
L,13356
L,289
L,13336
L,13337
L,26269
L,13323
L,26204
L,26214
L,13326
L,13085
L,13310
L,13281
L,13394
L,13395
L,26263
L,26265
L,13343
L,77
L,13273
L,13342
L,26271
L,11804
L,13322
L,13369
L,13353
L,26268
L,26201
L,26269
L,68
L,13391
''')

subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '10', '--gaussian',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois
Running: ciftify_surface_rois /scratch/edickie/ciftify_intergration_tests/src_data/vertices1.csv 10 --gaussian /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii


0

In [40]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     subid, 
     'gaussian_roi'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii sub-50005 gaussian_roi


0

In [46]:
run(['ciftify_peaktable', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_peaktable --max-threshold 0.05 --left-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii


0

In [42]:
vertices2_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices2_csv, "w") as text_file:
    text_file.write('''hemi,vertex
R,2379
R,2423
R,2423
R,2629
R,29290
R,29290
R,1794
R,29199
R,1788
R,2380
R,2288
R,29320
R,29274
R,29272
R,29331
R,29356
R,2510
R,29345
R,2506
R,2506
R,1790
R,29305
R,2630
R,2551
R,2334
R,2334
R,29306
R,2551
R,2422
R,29256
R,2468
R,29332
R,2425
R,29356
''')
    
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '6', '--probmap',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois
Running: ciftify_surface_rois /scratch/edickie/ciftify_intergration_tests/src_data/vertices1.csv 6 --probmap /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii


0

In [43]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     subid, 
     'probmap_roi'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii sub-50005 probmap_roi


0

In [47]:
run(['ciftify_peaktable', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_peaktable --max-threshold 0.05 --left-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii


0

In [50]:
for hemi in ['L','R']:
    run(['wb_command', '-surface-vertex-areas',
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness.32k_fs_LR.surf.gii'.format(subid, hemi)),
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness_va.32k_fs_LR.shape.gii'.format(subid, hemi))])

run(['ciftify_peaktable', 
    '--outputbase', os.path.join(new_outputs, 'rois', 'probmap_roi_withva'),
    '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
    '--left-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness_va.32k_fs_LR.shape.gii'.format(subid)),
    '--right-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness_va.32k_fs_LR.shape.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: wb_command -surface-vertex-areas /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness_va.32k_fs_LR.shape.gii
Running: wb_command -surface-vertex-areas /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness_va.32k_fs_LR.shape.gii
Running: ciftify_peaktable --outputbase /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi_withva --max-threshold 0.05 --left-surface /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /sc

0

In [51]:
run(['ciftify_surface_rois', '--debug',
      os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
      '6', '--vertex-col', 'tvertex', '--labels-col', 'NETWORK', '--overlap-logic', 'EXCLUDE',
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_surface_rois --debug /projects/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv 6 --vertex-col tvertex --labels-col NETWORK --overlap-logic EXCLUDE /projects/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /projects/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii


0

In [52]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     subid, 
     'tvertex'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii sub-50005 tvertex


0

# Running ciftify_seed_corr 

In [53]:
def seed_corr_default_out(func, seed):
    functype, funcbase = ciftify.io.determine_filetype(func)
    _, seedbase = ciftify.io.determine_filetype(seed)
    outputdir = os.path.dirname(func)
    outbase = '{}_{}'.format(funcbase, seedbase)
    outputname = os.path.join(outputdir, outbase)
    if functype == "nifti": outputname = '{}.nii.gz'.format(outputname)
    if functype == "cifti": outputname = '{}.dscalar.nii'.format(outputname)
    return(outputname)

In [54]:
def run_vis_map(result_map, result_prefix, result_type):
    run(['cifti_vis_map', '{}-snaps'.format(result_type), '--hcp-data-dir', hcp_data_dir, 
     result_map, subid, result_prefix])

In [55]:
def run_seedcorr_peaktable(result_map):
    run(['ciftify_peaktable',  
         '--min-threshold', '-0.5', 
         '--max-threshold', '0.5', 
         '--no-cluster-dlabel', 
         result_map])

In [56]:
subid = subids[0]
smoothing = 12
func_vol = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1.nii.gz')
func_cifti_sm0 = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s0.dtseries.nii')
func_cifti_smoothed = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s{}.dtseries.nii'.format(smoothing))
atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')

seed_corr_dir = os.path.join(new_outputs, 'ciftify_seed_corr')
if not os.path.exists(seed_corr_dir):
    run(['mkdir', '-p', seed_corr_dir])
struct = 'RIGHT-PUTAMEN'
putamen_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])

Running: mkdir -p /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr
Running: wb_command -volume-label-to-roi /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN


0

In [57]:
run(['ciftify_seed_corr', func_vol, putamen_vol_seed_mask])

Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [58]:
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_vol, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitonifti_unmasked'.format(subid, struct)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_sub-50005_RIGHT-PUTAMEN_vol.nii.gz sub-50005 sub-50005_RIGHT-PUTAMEN_niftitonifti_unmasked


0

In [59]:
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, putamen_vol_seed_mask])

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.nii.gz --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [60]:
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     subid, 
     '{}_{}_niftitonifti_masked'.format(subid, struct)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.nii.gz sub-50005 sub-50005_RIGHT-PUTAMEN_niftitonifti_masked


0

In [61]:
run(['ciftify_seed_corr', func_cifti_smoothed, putamen_vol_seed_mask])

Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [62]:
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitocifti_unmasked'.format(subid, struct)])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_unmasked


0

In [63]:
run_seedcorr_peaktable(seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask))

Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii


In [64]:
cifti_mask = os.path.join(seed_corr_dir, '{}_func_mask.dscalar.nii'.format(subid))
run(['wb_command', '-cifti-math', "'(x > 0)'", cifti_mask,
    '-var', 'x', func_cifti_sm0, '-select', '1', '1'])
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, putamen_vol_seed_mask])

Running: wb_command -cifti-math '(x > 0)' /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii -var x /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii -select 1 1
parsed '(x > 0)' as '(x>0)'

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [65]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     subid, 
     '{}_{}_niftitocifti_masked'.format(subid, struct)])
run_seedcorr_peaktable(os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)))

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii


In [66]:
result_map = seed_corr_default_out(func_cifti_smoothed, 
                                   os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct)))
result_prefix = '{}_{}_ciftitocifti_unmasked'.format(subid, struct)
result_type = 'cifti'

In [67]:
run(['wb_command', '-cifti-create-dense-from-template',
     func_cifti_sm0,
     os.path.join(seed_corr_dir, 
                                 '{}_{}_cifti.dscalar.nii'.format(subid, struct)),
     '-volume-all', putamen_vol_seed_mask])

run(['ciftify_seed_corr', func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

Running: wb_command -cifti-create-dense-from-template /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii -volume-all /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii


0

In [68]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii


In [69]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_masked.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_masked'.format(subid, struct)

In [70]:
run(['ciftify_seed_corr',     
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

Running: ciftify_seed_corr --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii


0

In [71]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_ciftitocifti_masked.dscalar.nii


In [72]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_unmasked'.format(subid)


In [73]:
run(['ciftify_seed_corr', '--weighted', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii
cmd: ciftify_seed_corr --weighted --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii 
 Failed with returncode 1
Usage:
    ciftify_seed_corr [options] <func> <seed>



1

In [74]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii 20

In [75]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_masked'.format(subid)

In [76]:
run(['ciftify_seed_corr', '--weighted', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii
cmd: ciftify_seed_corr --weighted --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ci

1

In [77]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii 20 20 COLUMN /tmp/tmpwsehv3_3/extrema.dscalar.nii -left-surface /projects/edickie/code/ciftify/c

In [78]:
L_gaussian_roi = os.path.join(new_outputs, 'rois', 'gaussian_L_roi.shape.gii')

result_map = seed_corr_default_out(func_cifti_smoothed, L_gaussian_roi)
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_unmasked'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_LEFT', L_gaussian_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     func_cifti_smoothed, 
     L_gaussian_roi])

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_roi.dscalar.nii COLUMN -metric CORTEX_LEFT /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_L_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi L /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_L_roi.shape.gii


0

In [79]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii


In [80]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)

In [81]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     '--outputname', os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     L_gaussian_roi])

Running: ciftify_seed_corr --weighted --hemi L --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/gaussian_L_roi.shape.gii


0

In [82]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii


In [83]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_unmasked.dscalar.nii'.format(subid)

In [84]:
run(['ciftify_seed_corr', '--weighted', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii
cmd: ciftify_seed_corr --weighted --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii 
 Failed with returncode 1
Usage:
    ciftify_seed_corr [options] <func> <seed>



1

In [85]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_unmasked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.

In [86]:
result_map = os.path.join(seed_corr_dir, '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid)

In [87]:
run(['ciftify_seed_corr', '--weighted', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii
cmd: ciftify_seed_corr --weighted --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify

1

In [88]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii 20 20 COLUMN /tmp/tmpfcq5l35c/extrema.dscalar.nii -left-surface /projects/edickie/code/ciftify

In [89]:
R_probmap_roi = os.path.join(new_outputs, 'rois', 'probmap_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_probmap_roi) 
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_unmasked.dscalar.nii'.format(subid)
                                   
run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_probmap_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', func_cifti_smoothed, 
     R_probmap_roi])

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_roi.dscalar.nii COLUMN -metric CORTEX_RIGHT /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_R_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_R_roi.shape.gii


0

In [90]:
result_map = os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)

In [91]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,    
     func_cifti_smoothed, 
     R_probmap_roi])

Running: ciftify_seed_corr --weighted --hemi R --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/probmap_R_roi.shape.gii


0

In [92]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii


In [93]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_unmasked.dscalar.nii'.format(subid)

In [94]:
run(['ciftify_seed_corr', '--roi-label', '7', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_seed_corr --roi-label 7 --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii
cmd: ciftify_seed_corr --roi-label 7 --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii 
 Failed with returncode 1
Usage:
    ciftify_seed_corr [options] <func> <seed>



1

In [95]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex.dscalar.nii sub-50005 sub-50005_tvertex7_ciftitocifti_unmasked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex.dscalar.nii 20 20 COLU

In [96]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid)

In [97]:
run(['ciftify_seed_corr', '--roi-label', '7', 
     '--outputname', result_map,
     '--mask', cifti_mask, 
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_seed_corr --roi-label 7 --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii
cmd: ciftify_seed_corr --roi-label 7 --surfaceonly --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/c

1

In [98]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii
cmd: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii 
 Failed with returncode 1
cmd: wb_command -cifti-extrema /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_ciftitocifti_masked.dscalar.nii 20 20 COLUMN /tmp/tmpogqoqu61/extrema.dscalar.nii -left-surface /projects/edickie/co

In [99]:
R_tvertex_roi = os.path.join(new_outputs, 'rois', 'tvertex_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_tvertex_roi)
result_type = 'cifti'
result_prefix = '{}_tvertex7_giftitocifti_umasked.dscalar.nii'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_tvertex_roi])
run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     func_cifti_smoothed, R_tvertex_roi])

Running: wb_command -cifti-separate /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii COLUMN -metric CORTEX_RIGHT /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex_R_roi.shape.gii
Running: ciftify_seed_corr --roi-label 7 --hemi R /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex_R_roi.shape.gii


0

In [100]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex_R_roi.dscalar.nii sub-50005 sub-50005_tvertex7_giftitocifti_umasked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_tvertex_R_roi.dscalar.nii


In [101]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)

In [102]:
run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_smoothed, R_tvertex_roi])

Running: ciftify_seed_corr --roi-label 7 --hemi R --outputname /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_giftitocifti_masked.dscalar.nii --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex_R_roi.shape.gii


0

In [103]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_tvertex7_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex7_giftitocifti_masked.dscalar.nii


# ciftify_meants (atlas examples)

In [104]:
subject_aparc = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.aparc.32k_fs_LR.dlabel.nii'.format(subid))
subject_thickness = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.thickness.32k_fs_LR.dscalar.nii'.format(subid))
run(['ciftify_meants', func_cifti_sm0, subject_aparc])


Running: ciftify_meants --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
cmd: ciftify_meants --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii 
 Failed with returncode 1
Usage:
    ciftify_meants [options] <func> <seed>



1

In [105]:
run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_aparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_sm0, subject_aparc])

Running: ciftify_meants --surfaceonly --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_func_ciftitocifti_masked_meants.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
cmd: ciftify_meants --surfaceonly --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_func_ciftitocifti_masked_meants.csv --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii

1

In [106]:
run(['ciftify_meants',  
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                    '{}_aparc_ciftitocifti_unmasked_labels.csv'.format(subid)),
         subject_thickness, subject_aparc])

Running: ciftify_meants --surfaceonly --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_ciftitocifti_unmasked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
cmd: ciftify_meants --surfaceonly --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_ciftitocifti_unmasked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-

1

In [107]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
         subject_thickness, subject_aparc])

Running: ciftify_meants --surfaceonly --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
cmd: ciftify_meants --surfaceonly --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thick

1

In [108]:
run(['ciftify_meants', func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii
cmd: ciftify_meants --surfaceonly /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii 
 Failed with returncode 1
Usage:
    ciftify_meants [options] <func> <seed>



1

In [109]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_tvertex_func_ciftitocifti_unmasked_meants.csv'.format(subid)),
     func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants --surfaceonly --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex_func_ciftitocifti_unmasked_meants.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/rois/tvertex.dscalar.nii
cmd: ciftify_meants --surfaceonly --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_tvertex_func_ciftitocifti_unmasked_meants.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/ru

1

In [110]:
## project wmparc to subject
wmparc_dscalar = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dscalar.nii'.format(subid))
run(['ciftify_vol_result', '--hcp-data-dir', hcp_data_dir,
    '--integer-labels', subid, atlas_vol, wmparc_dscalar])

Running: ciftify_vol_result --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp --integer-labels sub-50005 /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_MNI.dscalar.nii


0

In [111]:
run(['ciftify_meants', 
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, atlas_vol])

Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/wmparc.nii.gz
/projects/edickie/code/miniconda3/envs/ciftify_prereqs/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/projects/edickie/code/miniconda3/envs/ciftify_prereqs/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divid

0

In [112]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dscalar])

Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_MNI.dscalar.nii


0

In [113]:
wmparc_dlabel = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dlabel.nii'.format(subid))
run(['wb_command', '-cifti-label-import', '-logging', 'SEVERE',
     wmparc_dscalar, 
     os.path.join(ciftify.config.find_ciftify_global(), 'hcp_config', 'FreeSurferAllLut.txt'),
    wmparc_dlabel])
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dlabel])

Running: wb_command -cifti-label-import -logging SEVERE /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_MNI.dscalar.nii /projects/edickie/code/ciftify/ciftify/data/hcp_config/FreeSurferAllLut.txt /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_MNI.dlabel.nii
Running: ciftify_meants --mask /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_ciftiltocifti_masked_meants.csv --outputlabels /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-50005_wmparc_func_ciftiltocifti_masked_labels.csv /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/ciftify_seed_corr/sub-5

0

In [114]:
run(['cifti_vis_map', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_map index --hcp-data-dir /scratch/edickie/ciftify_intergration_tests/run_2017-09-01/hcp


0

In [ ]:
### All results
# ciftify_peaktable - with va input
# cifti_vis_map



In [ ]:
#group_nifti = '/home/edickie/Documents/ciftify_tests/src_data/working_memory_pFgA_z_FDR_0.01.nii.gz'
#group_cifti = '/home/edickie/Documents/ciftify_tests/run_2017-08-17/working_memory_pFgA_z_FDR_0.01.dscalar.nii'
#run(['ciftify_vol_result', '--debug', 'HCP_S1200_GroupAvg', group_nifti, group_cifti])

In [ ]:
recon_all_always = sorted(set(test3).intersection(set(test1)))

In [ ]:
onlytest2 = set(test2) - set(test1)
recon_all_lowres = [ x for x in onlytest2 if 'T1w' in x ]
sorted(recon_all_lowres)

In [ ]:
only_in_5 = set(test3) - set(test2)

In [ ]:
tmp1 = set(test2) - set(test3)
tmp1 = [ x for x in tmp1 if 'T1w' not in x ]
tmp1 = [ x for x in tmp1 if 'Results' not in x ]
tmp1 = [ x for x in tmp1 if 'aseg' not in x ]
only_in_6 = sorted(tmp1)
only_in_6

In [ ]:
ciftify_recon_all = { 'always': recon_all_always,
                    'T1w32k': recon_all_lowres,
                    'fs5' : only_in_5,
                    'fs6' : only_in_6}

In [ ]:
with open('/home/edickie/code/ciftify/tests/integration/data/ciftify_recon_all.yml', 'w') as outfile:
    yaml.dump(ciftify_recon_all, outfile, default_flow_style=False)

In [ ]:
with open('data.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)